In [1]:
#if necessary
!pip install BeautifulSoup4


     |████████████████████████████████| 112kB 5.0MB/s eta 0:00:01


In [8]:
#if necessary
#!pip install lxml


In [2]:
# if necessary
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.8           |   py36h45558ae_2         628 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_1        11.1 MB  conda-

In [3]:
# if necessary
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
import pandas as pd
import numpy as np
import requests 
#from bs4 import BeautifulSoup

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
creds = pd.read_csv('CREDS.csv')
CLIENT_ID = creds[creds['API']=="FOURSQUARE"].CID[0]
CLIENT_SECRET = creds[creds['API']=="FOURSQUARE"].SEC[0]
VERSION = '20180604'
LIMIT = 30

In [7]:
pcodes = pd.read_csv("KCPD_Crime_Data_2019.csv")
pcodes.head()

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,Beat,Address,City,Zip Code,Rep_Dist,Area,DVFlag,Involvement,Race,Sex,Age,Firearm Used Flag,Location
0,KC19019972,03/19/2019,08:48,03/19/2019,08:48,03/19/2019,09:33,Vehicular – Non-Injury,NaN,NaN,636,NE 40TH ST and N KIMBALL DR,KANSAS CITY,64161.0,PC1239,SCP,N,ARR CHA VDR,W,F,41.0,False,NaN
1,KC19020439,03/20/2019,17:11,03/20/2019,17:30,NaN,NaN,City/Municipal Warrant Arrest,NaN,NaN,999,00 S WATER ST,LIBERTY,64068.0,NaN,OSPD,N,ARR CHA SUS,W,M,31.0,False,NaN
2,KC19021179,03/23/2019,09:40,03/23/2019,09:40,NaN,NaN,Vehicular – Injury,NaN,NaN,423,N OAK TRFY and NE M 152 HWY E,KANSAS CITY,64155.0,PC0277,NPD,N,ARR CHA VDR,W,M,41.0,False,NaN
3,KC19021555,03/25/2019,07:35,03/25/2019,09:09,NaN,NaN,Possession/Sale/Distribution of a Controlled S...,35A,Drug / Narcotic Violations,115,00 W PERSHING RD,KANSAS CITY,64108.0,PJ1831,CPD,N,ARR CHA SUS,B,M,31.0,False,NaN
4,KC19021815,03/26/2019,06:04,03/26/2019,07:20,03/26/2019,08:00,Stealing – Shoplift,23C,Shoplifting,345,11600 E US 40 HWY,KANSAS CITY,64133.0,PJ3601,EPD,N,VIC,NaN,NaN,NaN,False,NaN


In [8]:
# only really interested in location and nature of crime
df = pcodes[['Location', 'Offense', 'IBRS', 'Description', 'DVFlag']]

print ('Rows: ' + str(df.shape[0]))
print ('-----')
print (df.count())
print ('Looks like 78046 rows. Offense appears to be the only key flag that will always be present')  
print ('But let\'s go ahead and drop all rows with nulls')
df = df.dropna()
df = df.reset_index()
print (df.count())
df.head()

Rows: 78046
-----
Location       64209
Offense        78046
IBRS           63103
Description    63103
DVFlag         78046
dtype: int64
Looks like 78046 rows. Offense appears to be the only key flag that will always be present
But let's go ahead and drop all rows with nulls
index          52775
Location       52775
Offense        52775
IBRS           52775
Description    52775
DVFlag         52775
dtype: int64


,index,Location,Offense,IBRS,Description,DVFlag
0,8,4600 S WORNALL RD\nKANSAS CITY 64112\n(39.043...,Embezzlement,270,Embezzlement,N
1,26,E 18 ST and WHITE AVE\nKANSAS CITY\n(39.089094...,Rape – Statutory,36B,Statutory Rape,N
2,27,"E 45 ST and LAWN AVE\nKANSAS CITY\n(39.043765,...",Domestic Violence Robbery (Strong-Armed),120,Robbery,Y
3,67,8200 NE UNDERGROUND DR\nKANSAS CITY 64119\n(3...,Assault (Non-Aggravated),13B,Simple Assault,N
4,112,"400 N PARK AVE\nKANSAS CITY\n(39.117113, -94....",Assault (Aggravated),13A,Aggravated Assault,N


In [9]:
print ('i want to extract those geo coordinates')
#import re
#df['Coordinates'] = re.search(, df['Location'])
#df['Coordinates'] = df['Location'].str[str(df['Location']).find('(') - 10:len(str(df['Location']))]
df['Coordinates'] = df['Location'].str[-22:-1]
#df.head()

#check = df.iloc[0]['Location']
check = df.iloc[0]['Coordinates']
#check = check.split(', ')
print(check) 
print('lat: ' + check.split(', ')[0] + ', lng: ' + check.split(', ')[1]) 

print('Not every row has valid coordinates so let\'s clean that up') 
df = df[df.Coordinates.str.contains(', ')]
df.count() 

print('And take care of parentheses')
df['Coordinates'] = df['Coordinates'].str.replace('(', '')
df['Coordinates'] = df['Coordinates'].str.replace(')', '')
df['Coordinates'] = df['Coordinates'].str.replace('n', '')
df['Coordinates'] = df['Coordinates'].str.replace('\n', '')
df.count() 

#df = df.reset_index()

df['Coordinates'] = df['Coordinates'].str.split(', ') 

df['Latitude'] = df['Coordinates'].str[0]
#df['Latitude'] = df['Coordinates'][0] 
#df['Latitude'] = df['Coordinates'].str.split(', ')[0] 
df['Longitude'] = df['Coordinates'].str[1]
df.head(5)

i want to extract those geo coordinates
39.043743, -94.589653
lat: 39.043743, lng: -94.589653
Not every row has valid coordinates so let's clean that up
And take care of parentheses


,index,Location,Offense,IBRS,Description,DVFlag,Coordinates,Latitude,Longitude
0,8,4600 S WORNALL RD\nKANSAS CITY 64112\n(39.043...,Embezzlement,270,Embezzlement,N,"[39.043743, -94.589653]",39.043743,-94.589653
1,26,E 18 ST and WHITE AVE\nKANSAS CITY\n(39.089094...,Rape – Statutory,36B,Statutory Rape,N,"[39.089094, -94.512695]",39.089094,-94.512695
2,27,"E 45 ST and LAWN AVE\nKANSAS CITY\n(39.043765,...",Domestic Violence Robbery (Strong-Armed),120,Robbery,Y,"[39.043765, -94.529939]",39.043765,-94.529939
3,67,8200 NE UNDERGROUND DR\nKANSAS CITY 64119\n(3...,Assault (Non-Aggravated),13B,Simple Assault,N,"[39.156368, -94.483751]",39.156368,-94.483751
4,112,"400 N PARK AVE\nKANSAS CITY\n(39.117113, -94....",Assault (Aggravated),13A,Aggravated Assault,N,"[39.117113, -94.554732]",39.117113,-94.554732


In [10]:
print ('Last sanity check remove non-numerics')
df = df[df.Longitude.str.match('-[0-9]{1,8}.[0-9]{1,8}')]
df = df[df.Latitude.str.match('^[0-9]{1,8}.[0-9]{1,8}')]
df.count()

Last sanity check remove non-numerics


index          52742
Location       52742
Offense        52742
IBRS           52742
Description    52742
DVFlag         52742
Coordinates    52742
Latitude       52742
Longitude      52742
dtype: int64

In [11]:
print ('We can filter out the domestic violence calls because that doesn\'t really apply here.')
df = df[df.DVFlag == 'N']
#print (df.count())

print ('Can we filter on just the violent and property crimes?')
print ('And really just the ones a bar patron might encounter.')
#print(df.groupby('Description')['index'].nunique())

violent_crimes = ['Aggravated Assault'
              ,'Forcible Rape'
              ,'Forcible Sodomy'
              ,'Kidnapping / Abduction'
              ,'Murder'
              ,'Robbery'
              ,'Sexual Assault With An Object'
              ,'Simple Assault']

df_violence = df[df.Description.isin(violent_crimes)]
print('Violent crimes: ')
print(df_violence.groupby('Description')['index'].nunique())

property_crimes = ['Motor Vehicle Theft'
              ,'Pocket-Picking'
              ,'Purse-Snatching'
              ,'Theft From Motor Vehicle'
              ,'Theft of Vehicle Parts and Accessories']

df_property = df[df.Description.isin(property_crimes)]
print('Property crimes: ')
print(df_property.groupby('Description')['index'].nunique())


We can filter out the domestic violence calls because that doesn't really apply here.
Can we filter on just the violent and property crimes?
And really just the ones a bar patron might encounter.
Violent crimes: 
Description
Aggravated Assault               3478
Forcible Rape                     208
Forcible Sodomy                   174
Kidnapping / Abduction             50
Murder                            218
Robbery                          2033
Sexual Assault With An Object       4
Simple Assault                   3008
Name: index, dtype: int64
Property crimes: 
Description
Motor Vehicle Theft                       3512
Pocket-Picking                              39
Purse-Snatching                             32
Theft From Motor Vehicle                  3298
Theft of Vehicle Parts and Accessories    1306
Name: index, dtype: int64


In [12]:
#Use geopy library to get the latitude and longitude values of Kansas City.

address = 'Kansas City, MO'
geolocator = Nominatim(user_agent="kc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#I'd like to reposition the map to focus on the entertainment areas
latitude = latitude - .03
longitude = longitude + .03
print('The geographical coordinates of KCMO are {}, {}.'.format(latitude, longitude))

The geographical coordinates of KCMO are 39.070105, -94.5481416.


In [33]:
#Zoom in on the commercial center of the city 
map_kcmo = folium.Map(location=[latitude, longitude], zoom_start=13) 
#map_kcmo

In [19]:
search_query = 'Bar'
near = 'Kansas City, MO'

for x in range(0, 4):
    print(x)
    offset = x * 50
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&near={}&query={}&intent=browse&offset={}&limit=50'.format(CLIENT_ID, CLIENT_SECRET, VERSION, near, search_query, offset)
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    if x == 0: 
        df_venues = json_normalize(venues)
    else: 
        df_temp = json_normalize(venues)
        df_venues = df_venues.append(df_temp, ignore_index = True)     
    print (df_venues.shape)
        
df_venues.head()


0


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


(50, 28)
1


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


(100, 28)
2
(150, 28)
3
(174, 28)


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,venue.photos.count,venue.photos.groups,venue.venuePage.id,flags.outsideRadius,venue.location.neighborhood
0,e-0-4ade13d6f964a520e87221e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4ade13d6f964a520e87221e3,The Well,7421 N Broadway St,at W 74th Ter,38.993441,-94.593661,"[{'label': 'display', 'lat': 38.99344080828442...",64118,US,Kansas City,MO,United States,"[7421 N Broadway St (at W 74th Ter), Kansas Ci...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",2054174,https://www.grubhub.com/restaurant/the-well-ba...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,0,[],54955022,NaN,NaN
1,e-0-4b5a3794f964a520f0b428e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b5a3794f964a520f0b428e3,Nick & Jake's,6325 Lewis Dr Ste 110,Rt.9,39.209586,-94.684092,"[{'label': 'display', 'lat': 39.20958572139344...",64152,US,Parkville,MO,United States,"[6325 Lewis Dr Ste 110 (Rt.9), Parkville, MO 6...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",2049649,https://www.grubhub.com/restaurant/nick-and-ja...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,0,[],NaN,NaN,NaN
2,e-0-4d4b70c29a8af04d69640daf-2,0,"[{'summary': 'This spot is popular', 'type': '...",4d4b70c29a8af04d69640daf,Al's Bar and Grill,11015 NW Highway 45,NaN,39.209305,-94.707750,"[{'label': 'display', 'lat': 39.20930457655886...",64152,US,Parkville,MO,United States,"[11015 NW Highway 45, Parkville, MO 64152, Uni...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,True,NaN
3,e-0-5531a384498ed68e40309d7a-3,0,"[{'summary': 'This spot is popular', 'type': '...",5531a384498ed68e40309d7a,Craft & Draft,"1 Royal Way, Kansas City, MO 64129",NaN,39.051942,-94.480904,"[{'label': 'display', 'lat': 39.05194152742662...",NaN,US,Kansas City,MO,United States,"[1 Royal Way, Kansas City, MO 64129, Kansas Ci...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN,NaN
4,e-0-4ad4c01df964a52076f120e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c01df964a52076f120e3,The Brick,1727 McGee St,btwn E 17th & 18th St,39.092318,-94.579952,"[{'label': 'display', 'lat': 39.09231771863946...",64108,US,Kansas City,MO,United States,"[1727 McGee St (btwn E 17th & 18th St), Kansas...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN,NaN


In [20]:
#results
#venues = results['response']['venues']
#df_venues = json_normalize(venues)

print (df_venues.shape)
df_venues.head()

(174, 28)


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,venue.photos.count,venue.photos.groups,venue.venuePage.id,flags.outsideRadius,venue.location.neighborhood
0,e-0-4ade13d6f964a520e87221e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4ade13d6f964a520e87221e3,The Well,7421 N Broadway St,at W 74th Ter,38.993441,-94.593661,"[{'label': 'display', 'lat': 38.99344080828442...",64118,US,Kansas City,MO,United States,"[7421 N Broadway St (at W 74th Ter), Kansas Ci...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",2054174,https://www.grubhub.com/restaurant/the-well-ba...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,0,[],54955022,NaN,NaN
1,e-0-4b5a3794f964a520f0b428e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b5a3794f964a520f0b428e3,Nick & Jake's,6325 Lewis Dr Ste 110,Rt.9,39.209586,-94.684092,"[{'label': 'display', 'lat': 39.20958572139344...",64152,US,Parkville,MO,United States,"[6325 Lewis Dr Ste 110 (Rt.9), Parkville, MO 6...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",2049649,https://www.grubhub.com/restaurant/nick-and-ja...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,0,[],NaN,NaN,NaN
2,e-0-4d4b70c29a8af04d69640daf-2,0,"[{'summary': 'This spot is popular', 'type': '...",4d4b70c29a8af04d69640daf,Al's Bar and Grill,11015 NW Highway 45,NaN,39.209305,-94.707750,"[{'label': 'display', 'lat': 39.20930457655886...",64152,US,Parkville,MO,United States,"[11015 NW Highway 45, Parkville, MO 64152, Uni...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,True,NaN
3,e-0-5531a384498ed68e40309d7a-3,0,"[{'summary': 'This spot is popular', 'type': '...",5531a384498ed68e40309d7a,Craft & Draft,"1 Royal Way, Kansas City, MO 64129",NaN,39.051942,-94.480904,"[{'label': 'display', 'lat': 39.05194152742662...",NaN,US,Kansas City,MO,United States,"[1 Royal Way, Kansas City, MO 64129, Kansas Ci...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN,NaN
4,e-0-4ad4c01df964a52076f120e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c01df964a52076f120e3,The Brick,1727 McGee St,btwn E 17th & 18th St,39.092318,-94.579952,"[{'label': 'display', 'lat': 39.09231771863946...",64108,US,Kansas City,MO,United States,"[1727 McGee St (btwn E 17th & 18th St), Kansas...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN,NaN


In [ ]:
for lat, lng, label in zip(df_venues['venue.location.lat'], df_venues['venue.location.lng'], df_venues['venue.name']):
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        popup=label,
        radius=5,
        color='black',
        fill = True,
        fill_color='black',
        fill_opacity=0.2,
        parse_html=False).add_to(map_kcmo)
map_kcmo

In [ ]:
# add violent crime markers to map
for lat, lng, label in zip(df_violence['Latitude'].astype(float), df_violence['Longitude'].astype(float), df_violence['Offense']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.1,
        parse_html=False).add_to(map_kcmo)
map_kcmo

In [ ]:
# add property crime markers to map
for lat, lng, label in zip(df_property['Latitude'].astype(float), df_property['Longitude'].astype(float), df_property['Offense']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.1,
        parse_html=False).add_to(map_kcmo) 
map_kcmo